In [257]:
import numpy as np
import gurobipy as gp
from gurobipy import *
from numpy import *

In [308]:
class SubProblem:
    def __init__(self, lengths, W) -> None:
        self.lengths, self.M, self.W = lengths, len(lengths), W
    
    def create_model(self):
        self.model = gp.Model("sub model")
        self.y = self.model.addVars(self.M, lb = 0, ub = GRB.INFINITY, vtype = GRB.INTEGER, name = 'y')
        self.model.addConstr((gp.quicksum(self.lengths[i]*self.y[i] for i in range(self.M)) <= self.W))
    
    def set_objective(self, pi):
        self.model.setObjective(gp.quicksum(pi[i]*self.y[i] for i in range(self.M)), sense = GRB.MAXIMIZE)
    
    def solve(self, flag = 0):
        self.model.Params.OutputFlag = flag
        self.model.optimize()
    
    def get_solution(self):
        return [self.model.getVars()[i].x for i in range(self.M)]

    def get_reduced_cost(self):
        return self.model.ObjVal
    
    def write(self):
        self.model.write("sub_model.lp")


In [417]:
#test by hand
# W = 15 #Total length of one roll
# lengths = [4, 6, 7]
# demands = [80, 50, 100]

#read from mps
m = gp.read("cutting_stock_model.mps")
x = m.getVars()
i =0
#find the var L and its val
for cnstr in m.getConstrs():
    i+=1
    if(cnstr.sense == "="):
        W = cnstr.RHS
        i -= 1
        break
#get rid of paramters related to L
a = m.getAttr("OBJ")
del a[i]
k = m.getAttr("RHS")
del k[i]
lengths = a
demands = k
M = len(lengths) # number of different cutting length
N = sum(demands) # number of available rolls
#bulid matrix in reviesd simplex method
a1 = [W/x for x in lengths]
B=mat(diag(a1), dtype = int);
cB=mat(ones((1,M)));
b = mat(demands).T
#import subproblem
sub_prob = SubProblem(lengths, W)
sub_prob.create_model()
iter = 0

while True:
    iter += 1
    print("Iteration:"+str(iter))
    y = cB*B.I
    x = B.I*b
    z = sum(x[:,:])
    pi = y.getA()[0]#transform matrix to array
    sub_prob.set_objective(pi)
    sub_prob.solve()
    pattern = sub_prob.get_solution()
    pattern = [abs(x) for x in pattern]
    reduced_cost = sub_prob.get_reduced_cost()
    print("new cutting pattern:" + str(pattern))
    if reduced_cost <= 1:
        break
    #update simplex
    entering = mat([pattern]).T
    decision = B.I*entering
    d_left = x.getA()
    d_right = decision.getA()
    curmin = GRB.INFINITY
    # find leaving var
    for i in range(len(d_right)):
        if(d_right[i] > 0):
            if(d_left[i]/d_right[i] < curmin):
                leaving_index = i
                curmin = d_left[i]/d_right[i]
    #reform the B matrix
    B_left=B[:,:leaving_index]
    B_right = B[:,leaving_index + 1:]
    tmp=hstack((B_left,entering))
    B = hstack((tmp,B_right))

print("Optimail cutting pattern:")
print(B)
print("Optimal using times of each cutting pattern" + str(x))
print("minimum cost of rolls:" + str(z))
print("Total iteration:" + str(iter))

Read MPS format model from file cutting_stock_model.mps
Reading time = 0.00 seconds
cs: 11 rows, 11 columns, 11 nonzeros
Iteration:1
new cutting pattern:[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]
Iteration:2
new cutting pattern:[1.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
Iteration:3
new cutting pattern:[0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
Iteration:4
new cutting pattern:[1.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0]
Iteration:5
new cutting pattern:[7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
Iteration:6
new cutting pattern:[0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
Iteration:7
new cutting pattern:[0.0, 1.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Iteration:8
new cutting pattern:[0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0]
Iteration:9
new cutting pattern:[4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
Iteration:10
new cutting pattern:[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
Iteration:11
new cutting pattern:[0.0, 2.0, 0.0,